In [82]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.mixture import GaussianMixture
import joblib
from tqdm import tqdm
import pandas as pd
import argparse
import os
from scipy.stats import multivariate_normal

In [83]:
# gmm recovery from a synthetic gmm
# make a representative gmm
def make_gmm(n_components, n_features, random_state=0):
    gmm = GaussianMixture(n_components=n_components, random_state=random_state)
    gmm.means_ = np.random.rand(n_components, n_features)
    covariance_array = []
    for i in range(n_components):
        rand_matrix = np.random.rand(n_features, n_features)
        covariance_array.append(np.dot(rand_matrix, rand_matrix.T))

    precision_array = [np.linalg.pinv(covariance_array[i]) for i in range(n_components)]
    gmm.precisions_ = np.array(precision_array)
    gmm.precisions_cholesky_ = np.array([np.linalg.cholesky(precision_array[i]) for i in range(n_components)])
    gmm.covariances_ = np.array(covariance_array)
    
    gmm.weights_ = np.random.rand(n_components)
    gmm.weights_ /= np.sum(gmm.weights_)
    return gmm

In [84]:
n_components = 5
n_features = 20

my_test_gmm = make_gmm(n_components, n_features)
my_test_gmm.sample(1)

(array([[-5.44149323, -4.04280332, -3.89185993, -3.79538471, -2.332354  ,
         -3.56951212, -4.89904712, -4.49417209, -2.63450021, -3.21219345,
         -2.07171713, -4.50845398, -1.29492845, -2.97777404, -5.18532151,
         -5.57904983, -4.9643353 , -5.40788313, -4.37305521, -2.66455146]]),
 array([4]))

In [85]:
# take compressive meaurements
# make 10 random measurement matrices
n_measurements = ((n_features * 4) // 10) # 40% of the features

measurement_matrices = []
for i in range(10):
    measurement_matrices.append(np.random.rand(n_measurements, n_features))

# generate samples from the gmm
n_samples = 1000
samples = my_test_gmm.sample(n_samples)

# take measurements
compressed_measurements = []
sample_measurement_matrix = []
for i in range(n_samples):
    matrix_index = np.random.randint(0, len(measurement_matrices))
    sample_measurement_matrix.append(matrix_index)
    compressed_measurements.append(np.dot(measurement_matrices[matrix_index], samples[0][i]))


noise_std_dev = 0.1
noise_covariance_matrix = np.eye(n_measurements) * noise_std_dev * noise_std_dev
compressed_measurements = np.array(compressed_measurements)
compressed_measurements += np.random.normal(0, noise_std_dev, compressed_measurements.shape)

In [86]:
# for each of the 10 measurement matrices we have a separate gmm in the y-domain so let us make all of them

def form_y_gmms(x_gmm, measurement_matrices, noise_covariance_matrix):
    gmm_list = []
    noise_inverse = np.linalg.inv(noise_covariance_matrix)
    for i in range(len(measurement_matrices)):
        gmm_sample = GaussianMixture(n_components=n_components, random_state=0)
        gmm_sample.means_ = x_gmm.means_ @ measurement_matrices[i].T
        covariance_array = []
        for j in range(n_components):
            covariance_array.append(np.linalg.pinv(noise_inverse - ((noise_inverse @ measurement_matrices[i]) @ np.linalg.pinv((measurement_matrices[i].T @ noise_inverse @ measurement_matrices[i]) + np.linalg.pinv(x_gmm.covariances_[j])) @ measurement_matrices[i].T @ noise_inverse)))

        gmm_sample.covariances_ = np.array(covariance_array)
        gmm_sample.weights_ = x_gmm.weights_

        precision_array = [np.linalg.pinv(gmm_sample.covariances_[i]) for i in range(n_components)]
        gmm_sample.precisions_ = np.array(precision_array)
        gmm_sample.precisions_cholesky_ = np.array([np.linalg.cholesky(precision_array[i]) for i in range(n_components)])
        
        gmm_list.append(gmm_sample)
    return gmm_list


In [87]:
def get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, gmm_list):
    log_likelihood = 0
    for i in range(len(compressed_measurements)):
        # print(gmm_list[sample_measurement_matrix[i]].score(compressed_measurements[i].reshape(1, -1)))
        log_likelihood += gmm_list[sample_measurement_matrix[i]].score(compressed_measurements[i].reshape(1, -1))

    return log_likelihood/len(compressed_measurements)

In [88]:
# initialise estimate of the original gmm
# assumption is that we know the number of components required to model the original gmm
estimated_gmm = make_gmm(n_components, n_features, random_state=10)

# perform updates on the estimated gmm
# log likelihoods stores the log likelihoods of the estimated gmm computed on the samples drawn from the original gmm, our objective is to maximise this
tol = 1e-3
log_likelihoods = []
log_likelihoods.append(get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, form_y_gmms(estimated_gmm, measurement_matrices, noise_covariance_matrix)))

print("Initial log likelihood: ", log_likelihoods[-1])
print("Ideal log likelihood: ", get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, form_y_gmms(my_test_gmm, measurement_matrices, noise_covariance_matrix)))

noise_inverse =  np.linalg.inv(noise_covariance_matrix)
y_gmms = form_y_gmms(estimated_gmm, measurement_matrices, noise_covariance_matrix)

while True:
    # perform updates
    # for each of the 10 measurement matrices we have a separate gmm in the y-domain so let us make all of them
    # update weights
    new_weights = []
    for i in range(n_components):
        ssum = 0
        for j in range(len(compressed_measurements)):
            den = 0
            for k in range(n_components):
                den += y_gmms[sample_measurement_matrix[j]].weights_[k] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[k], cov=y_gmms[sample_measurement_matrix[j]].covariances_[k])
            ssum += y_gmms[sample_measurement_matrix[j]].weights_[i] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[i], cov=y_gmms[sample_measurement_matrix[j]].covariances_[i]) / den
        new_weights.append(ssum)
    new_weights = np.array(new_weights)
    # new_weights /= np.sum(new_weights) # do this after mean and covariance update

    # update means
    new_means = []
    for i in range(n_components):
        ssum = 0
        for j in range(len(compressed_measurements)):
            den = 0
            for k in range(n_components):
                den += y_gmms[sample_measurement_matrix[j]].weights_[k] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[k], cov=y_gmms[sample_measurement_matrix[j]].covariances_[k])
            p_ik = y_gmms[sample_measurement_matrix[j]].weights_[i] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[i], cov=y_gmms[sample_measurement_matrix[j]].covariances_[i]) / den
            n_ik = (estimated_gmm.means_[i] + np.linalg.pinv((measurement_matrices[sample_measurement_matrix[j]].T @ noise_inverse @ measurement_matrices[sample_measurement_matrix[j]]) + estimated_gmm.precisions_[i]) @ (measurement_matrices[sample_measurement_matrix[j]].T @ noise_inverse @ (compressed_measurements[j] - y_gmms[sample_measurement_matrix[j]].means_[i])))
            ssum += p_ik * n_ik
        new_means.append(ssum / new_weights[i])

    # update covariances
    new_covariances = []
    for i in range(n_components):
        ssum = 0
        for j in range(len(compressed_measurements)):
            den = 0
            for k in range(n_components):
                den += y_gmms[sample_measurement_matrix[j]].weights_[k] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[k], cov=y_gmms[sample_measurement_matrix[j]].covariances_[k])
            p_ik = y_gmms[sample_measurement_matrix[j]].weights_[i] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[i], cov=y_gmms[sample_measurement_matrix[j]].covariances_[i]) / den
            C_ik = np.linalg.pinv((measurement_matrices[sample_measurement_matrix[j]].T @ noise_inverse @ measurement_matrices[sample_measurement_matrix[j]]) + estimated_gmm.precisions_[i])
            n_ik = estimated_gmm.means_[i] + C_ik @ (measurement_matrices[sample_measurement_matrix[j]].T @ noise_inverse @ (compressed_measurements[j] - y_gmms[sample_measurement_matrix[j]].means_[i]))
            ssum += p_ik * ((n_ik - new_means[i]) @ (n_ik - new_means[i]).T + C_ik)
        new_covariances.append(ssum / new_weights[i])

    estimated_gmm.weights_ = new_weights / np.sum(new_weights)
    estimated_gmm.means_ = np.array(new_means)
    estimated_gmm.covariances_ = np.array(new_covariances)
    estimated_gmm.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(estimated_gmm.covariances_))
    estimated_gmm.precisions_ = np.linalg.inv(estimated_gmm.covariances_)

    y_gmms = form_y_gmms(estimated_gmm, measurement_matrices, noise_covariance_matrix)
    log_likelihoods.append(get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, y_gmms))
    print(log_likelihoods[-1])
    if abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol:
        break
    pass


Initial log likelihood:  -18.57757356820687
Ideal log likelihood:  -17.644096683594977
-22.391154379786933
-28.899916278571755
-39.737552662789085
-55.72151231756294
-77.15513159736045
-103.64957595138159


KeyboardInterrupt: 